# Trader Behaviour & Market Sentiment Analysis (Complete)

This notebook is simple and beginner-friendly. Run cells sequentially. All visuals will be saved to `visual_insights/` and textual insights to `insights_summary.txt`.

In [ ]:
# Imports
import pandas as pd, numpy as np, matplotlib.pyplot as plt, os, math
from datetime import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
%matplotlib inline


In [ ]:
# Load data files (they must be in the same folder as this notebook)
sentiment_df = pd.read_csv('market_sentiment.csv', low_memory=False)
trader_df = pd.read_csv('trader_behaviour.csv', low_memory=False)

# clean column names
sentiment_df.columns = [str(c).strip().lower().replace(' ', '_') for c in sentiment_df.columns]
trader_df.columns = [str(c).strip().lower().replace(' ', '_') for c in trader_df.columns]

print('Sentiment columns:', sentiment_df.columns.tolist())
print('Trader columns:', trader_df.columns.tolist())

display(sentiment_df.head())
display(trader_df.head())

# create visual_insights folder
os.makedirs('visual_insights', exist_ok=True)


In [ ]:
# Merge datasets on floored date (daily merge)
date_cols_sent = [c for c in sentiment_df.columns if 'timestamp' in c or 'date' in c or 'time' in c]
date_cols_trad = [c for c in trader_df.columns if 'timestamp' in c or 'date' in c or 'time' in c]

if date_cols_sent:
    sentiment_df['_dt'] = pd.to_datetime(sentiment_df[date_cols_sent[0]], errors='coerce')
else:
    sentiment_df['_dt'] = pd.NaT
if date_cols_trad:
    trader_df['_dt'] = pd.to_datetime(trader_df[date_cols_trad[0]], errors='coerce')
else:
    trader_df['_dt'] = pd.NaT

sentiment_df['_date'] = sentiment_df['_dt'].dt.floor('d')
trader_df['_date'] = trader_df['_dt'].dt.floor('d')

merged_df = pd.merge(trader_df, sentiment_df, left_on='_date', right_on='_date', how='left', suffixes=('_trader','_sent'))
print('Merged shape:', merged_df.shape)
display(merged_df.head())

In [ ]:
# Visualizations (each plot is saved to visual_insights/)
# Detect numeric sentiment column
sentiment_col = None
for c in sentiment_df.select_dtypes(include=[np.number]).columns:
    if 'sentiment' in c:
        sentiment_col = c; break
if sentiment_col is None:
    if 'value' in sentiment_df.columns and pd.api.types.is_numeric_dtype(sentiment_df['value']):
        sentiment_col = 'value'
    else:
        num_cols = sentiment_df.select_dtypes(include=[np.number]).columns.tolist()
        sentiment_col = num_cols[0] if num_cols else None

# Detect target column
target_col = None
for cand in ['closed_pnl','pnl','profit','value']:
    if cand in merged_df.columns:
        target_col = cand; break
if target_col is None:
    numeric_candidates = [c for c in trader_df.select_dtypes(include=[np.number]).columns if trader_df[c].nunique() > 1]
    target_col = numeric_candidates[-1] if numeric_candidates else None

# Create 'profitable' boolean if possible
if target_col and pd.api.types.is_numeric_dtype(merged_df[target_col]):
    merged_df['profitable'] = merged_df[target_col] > 0
else:
    merged_df['profitable'] = np.nan

# Helper to save plots
def save_plot(path):
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

# 1) PnL distribution
if target_col and target_col in merged_df.columns and pd.api.types.is_numeric_dtype(merged_df[target_col]):
    data = merged_df[target_col].dropna()
    if len(data)>0:
        plt.figure(figsize=(8,5))
        lower, upper = np.percentile(data,1), np.percentile(data,99)
        plt.hist(data.clip(lower, upper), bins=80)
        plt.title('Distribution of closed PnL (1-99 pct clipped)')
        plt.xlabel('closed_pnl'); plt.ylabel('count')
        save_plot('visual_insights/pnl_distribution.png')
        display(plt.imread('visual_insights/pnl_distribution.png'))

# 2) Sentiment trend
if sentiment_col and sentiment_col in sentiment_df.columns:
    s_daily = sentiment_df.groupby(sentiment_df['_date'])[sentiment_col].mean().dropna()
    if len(s_daily)>0:
        plt.figure(figsize=(10,4))
        s_daily.plot()
        plt.title('Sentiment trend (daily mean)'); plt.xlabel('date'); plt.ylabel(sentiment_col)
        save_plot('visual_insights/sentiment_trend.png')
        display(plt.imread('visual_insights/sentiment_trend.png'))

# 3) Performance vs Sentiment scatter
if sentiment_col and sentiment_col in merged_df.columns and target_col and target_col in merged_df.columns:
    sub = merged_df[[sentiment_col, target_col]].dropna()
    if len(sub)>0:
        if len(sub)>20000: sub = sub.sample(20000, random_state=42)
        plt.figure(figsize=(7,5))
        plt.scatter(sub[sentiment_col], sub[target_col], alpha=0.3, s=10)
        plt.title('Performance vs Sentiment (scatter)'); plt.xlabel(sentiment_col); plt.ylabel(target_col)
        save_plot('visual_insights/performance_vs_sentiment.png')
        display(plt.imread('visual_insights/performance_vs_sentiment.png'))

# 4) Correlation heatmap (numeric only)
numeric_df = merged_df.select_dtypes(include=['number'])
if not numeric_df.empty:
    corr = numeric_df.corr()
    plt.figure(figsize=(10,8))
    plt.imshow(corr, aspect='auto')
    plt.colorbar(); plt.title('Correlation matrix (numeric cols)')
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90, fontsize=8)
    plt.yticks(range(len(corr.index)), corr.index, fontsize=8)
    save_plot('visual_insights/correlation_heatmap.png')
    display(plt.imread('visual_insights/correlation_heatmap.png'))

# 5) Trade side distribution
if 'side' in merged_df.columns:
    vc = merged_df['side'].fillna('unknown').value_counts()
    if len(vc)>0:
        plt.figure(figsize=(6,4))
        vc.plot(kind='bar')
        plt.title('Trade Side Distribution'); plt.xlabel('side'); plt.ylabel('count')
        save_plot('visual_insights/trade_side_distribution.png')
        display(plt.imread('visual_insights/trade_side_distribution.png'))

# 6) Average performance per sentiment quartile
if sentiment_col and sentiment_col in merged_df.columns and target_col and target_col in merged_df.columns:
    tmp = merged_df[[sentiment_col, target_col]].dropna()
    try:
        if tmp[sentiment_col].nunique()>1:
            tmp['sent_q'] = pd.qcut(tmp[sentiment_col], 4, labels=False, duplicates='drop')
            grp = tmp.groupby('sent_q')[target_col].mean()
            plt.figure(figsize=(6,4))
            grp.plot(kind='bar'); plt.title('Average performance per sentiment quartile (0=low)')
            plt.xlabel('sentiment_quartile'); plt.ylabel(f'avg {target_col}')
            save_plot('visual_insights/avg_perf_per_sentiment_quartile.png')
            display(plt.imread('visual_insights/avg_perf_per_sentiment_quartile.png'))
    except Exception as e:
        print('Could not compute quartiles:', e)


In [ ]:
# Simple models (optional) - regression on closed_pnl and classification on 'profitable'
# Prepare numeric features (drop ids and timestamps)
feature_df = merged_df.select_dtypes(include=['number']).copy()
for c in ['_date','_dt']:
    if c in feature_df.columns: feature_df = feature_df.drop(columns=[c])
# drop target from features
targets = [c for c in ['closed_pnl','pnl','profit','value'] if c in merged_df.columns]
target_col = targets[0] if targets else None

if target_col and target_col in merged_df.columns:
    if target_col in feature_df.columns: feature_df = feature_df.drop(columns=[target_col])
# keep columns with variation
feature_df = feature_df.loc[:, feature_df.nunique() > 1]

# Regression
if target_col and target_col in merged_df.columns and pd.api.types.is_numeric_dtype(merged_df[target_col]) and feature_df.shape[1]>0:
    df_reg = merged_df[[target_col]].join(feature_df).dropna()
    if len(df_reg) > 200:
        y = df_reg[target_col].values
        X = df_reg.drop(columns=[target_col]).values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        lr = LinearRegression(); lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        print('Regression MSE:', mean_squared_error(y_test, y_pred), 'R2:', r2_score(y_test, y_pred))
        # save scatter plot
        plt.figure(figsize=(6,5)); plt.scatter(y_test, y_pred, alpha=0.3, s=10); plt.xlabel('Actual'); plt.ylabel('Predicted'); plt.title('Linear Regression: Actual vs Pred')
        plt.tight_layout(); plt.savefig('visual_insights/regression_actual_vs_predicted.png'); plt.show()
    else:
        print('Not enough data for regression (need >200 rows after dropna).')

# Classification on 'profitable'
if 'profitable' in merged_df.columns and merged_df['profitable'].notna().any() and feature_df.shape[1]>0:
    df_clf = merged_df[['profitable']].join(feature_df).dropna()
    if len(df_clf) > 200 and df_clf['profitable'].nunique() > 1:
        y = df_clf['profitable'].astype(int).values
        X = df_clf.drop(columns=['profitable']).values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        clf = LogisticRegression(max_iter=1000); clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print('Classification accuracy:', accuracy_score(y_test, y_pred))
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        fig, ax = plt.subplots(figsize=(5,4)); disp.plot(ax=ax); plt.title('Logistic Regression Confusion Matrix')
        plt.tight_layout(); plt.savefig('visual_insights/clf_confusion_matrix.png'); plt.show()
    else:
        print('Not enough data for classification or only one class present.')

In [ ]:
# Textual insights (auto-save)
insights = []
insights.append('Report generated: ' + datetime.utcnow().isoformat() + ' UTC')
insights.append('Basic dataset sizes:')
insights.append(' - market_sentiment rows: ' + str(len(sentiment_df)))
insights.append(' - trader_behaviour rows: ' + str(len(trader_df)))
insights.append(' - merged rows: ' + str(len(merged_df)))
insights.append('')

# detect sentiment and target
sent_cols = [c for c in sentiment_df.select_dtypes(include=[np.number]).columns]
sentiment_col = None
for c in sent_cols:
    if 'sentiment' in c:
        sentiment_col = c; break
if sentiment_col is None:
    if 'value' in sentiment_df.columns and pd.api.types.is_numeric_dtype(sentiment_df['value']):
        sentiment_col = 'value'
    else:
        sentiment_col = sent_cols[0] if sent_cols else None

targets = [c for c in ['closed_pnl','pnl','profit','value'] if c in merged_df.columns]
target_col = targets[0] if targets else None

insights.append('Detected sentiment column: ' + str(sentiment_col))
insights.append('Detected performance column: ' + str(target_col))
insights.append('')

# target stats
if target_col and target_col in merged_df.columns and pd.api.types.is_numeric_dtype(merged_df[target_col]):
    desc = merged_df[target_col].describe()
    insights.append('Summary stats for ' + target_col + ':')
    for k in ['count','mean','std','min','25%','50%','75%','max']:
        if k in desc.index:
            insights.append(f'  {k}: {desc[k]}')
    insights.append('')

# sentiment vs performance
if sentiment_col and sentiment_col in merged_df.columns and target_col and target_col in merged_df.columns:
    valid = merged_df[[sentiment_col, target_col]].dropna()
    if len(valid)>0 and valid[sentiment_col].nunique()>1:
        corr_val = valid[sentiment_col].corr(valid[target_col])
        insights.append('Correlation between ' + sentiment_col + ' and ' + target_col + ': ' + str(round(corr_val,4)))
        try:
            q75 = valid[sentiment_col].quantile(0.75); q25 = valid[sentiment_col].quantile(0.25)
            top_mean = valid[valid[sentiment_col] >= q75][target_col].mean()
            bot_mean = valid[valid[sentiment_col] <= q25][target_col].mean()
            insights.append('Average ' + target_col + ' in top sentiment quartile: ' + str(top_mean))
            insights.append('Average ' + target_col + ' in bottom sentiment quartile: ' + str(bot_mean))
            if not math.isclose(bot_mean, 0.0):
                pct_change = ((top_mean - bot_mean) / abs(bot_mean)) * 100
                insights.append('Percent change (top vs bottom): ' + str(round(pct_change,2)) + '%')
            else:
                insights.append('Percent change (top vs bottom): undefined (bottom mean is ~0)')
        except Exception as e:
            insights.append('Could not compute quartile comparison: ' + str(e))
    else:
        insights.append('Not enough variation in sentiment to compute correlation/quartiles.')
else:
    insights.append('Sentiment or performance column missing; skipping relationship insights.')

# profitable counts
if 'profitable' in merged_df.columns and merged_df['profitable'].notna().any():
    prof_counts = merged_df['profitable'].value_counts(dropna=True)
    insights.append('Profitable trade counts:')
    for k,v in prof_counts.items():
        insights.append(f'  {k}: {v}')

with open('insights_summary.txt','w',encoding='utf-8') as f:
    f.write('\n'.join(insights))

print('Saved insights_summary.txt and correlation_matrix.csv (if numeric columns present)')

## Conclusion

All visuals are saved to `visual_insights/`. Insights are saved to `insights_summary.txt`. Run each cell sequentially.